# Employee Attrition Prediction Model

The model uses Employee Attrition dataset from Kaggle's and compares the effectiveness between using Logistic Regression, Neural Network, and Random Forest. We start by analyzing the dataset, keeping the significant ones and removing the uncorrelated ones to optimize performance from our models. Then we proceed to parsing, splitting, and normalizing before training our models. Below are explanations to our work and our code.

In [1]:
import pandas as pd

## Data Parsing

We start with all the data given from the dataset. The raw dataset has 1470 rows and 35 features including Attrition, which is our desired output feature. In terms of data types, there are 26 numerical features and 9 categorical features, ranging from 2 to 6 categories each.

The Parse function below takes in a .CSV file, and does the following things:
1. drop the constant features (columns_to_drop_1)
2. drop the redundant features (columns_to_drop_2)
3. split into input and output features
4. One-hot encode the categorical columns
5. Map the 'Attrition' feature into boolean values

In [ ]:
def parse_csv(file_path):
    """
    Parses the CSV file, normalizes numerical data, and one-hot encodes categorical variables.

    Parameters:
    - file_path: str, path to the CSV file.

    Returns:
    - X_processed: pandas DataFrame, processed feature data.
    - y_encoded: pandas Series, encoded target variable.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Dropping constant and redundant features
    columns_to_drop_1 = ['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber']
    columns_to_drop_2 = ['MonthlyIncome', 'TotalWorkingYears', 'YearsInCurrentRole', 'YearsWithCurrManager']
    df = df.drop(columns=columns_to_drop_1)
    df = df.drop(columns=columns_to_drop_2)

    # Separate features and target variable
    X = df.drop('Attrition', axis=1)
    y = df['Attrition']

    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # One-hot encode categorical variables
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

    # Doesn't normalize. Leave normalization until after splitting training, validation, and test sets.
    
    # Encode the target variable
    y_encoded = y.map({'Yes': 1, 'No': 0})

    return X_encoded, y_encoded

Note that the data isn't normalized when parsing. We leave normalization of data to after we split the data into training, validation, and testing sets. This ensures that there are no data leakages, and provides a better evaluation of our models' performance.

**Simple logistic regression**

**Prelminaries**

1. Load the data
2. Extract test data
3. Normalize the training data

**Simple logistic regression**

5. Train simple logistic regression
6. Evaluate the models with cross-validation

**Regularization**

7. LASSO regularized logistic regression
8. Choose the best model
9. Significant features
10. Evaluate the final model with test data

**Conclusion**
11. Compare and Contrast resulting accuracies
